In [1]:
!curl -o input.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  1899k      0 --:--:-- --:--:-- --:--:-- 1900k


In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print('Vocab size : ', vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size :  65


### Tokenisation

We convert a sequence of strings to sequence of integers according to some vocabulary mapping.

Here we are building a character level tokenizer, which means each character is mapped to an integer.

In [5]:
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for s, i in stoi.items()}
encode = lambda s: [stoi.get(c) for c in s] # s -> string, c -> a char in that string, s
decode = lambda l: ''.join(itos.get(i) for i in l) # list of ints to string

In [6]:
print(encode("hi"))
print(decode(encode("hi")))

[46, 47]
hi


In [7]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [8]:
# train val split

n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

print(len(train_data), len(val_data))

1003854 111540


In [9]:
torch.manual_seed(1137)
batch_size = 4 # how many independent sequence we process in parallel?
block_size = 8 # What is the maximum context length for prediction?

def get_batch(split):
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data)-block_size, (batch_size, ))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x, y


In [10]:
torch.randint(len(data) - block_size, (batch_size, ))

tensor([1074518,  726227, 1092993,  642395])

In [11]:
xb, yb = get_batch(train_data)

### Bigram Language Model

In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets):

        # idx and targets are both (B, T) tensors of integers
        # Batch, Time, Channel
        # Batch -> 4
        # Time -> Block size -> 8
        # Channel -> embedding vector size -> Vocab size
        logits = self.token_embedding_table(idx) # (B, T, C)

        loss = F.cross_entropy(logits, targets)

        return logits, loss

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)


RuntimeError: Expected target size [4, 65], got [4, 8]

### Self Attention

In [13]:
torch.manual_seed(1337)
B, T, C = 4, 8 , 2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])